# Simulations démographique

Quel phénomène a l'effet le plus important dans l'évolution proche (2024 - 2040) du taux de dépendance des personnes âgées ?
- le vieillissement des générations du babyboom (hypothèse "Laure")
- l'allongement de l'espérance de vie depuis les années 2000 (hypothèse "Geoffrey")

Pour cela on compare trois scénarios :
- le scénario central des projections de population de l'Insee pour 2070
- un scénario "sans baby boom", c'est à dire où l'évolution de la population est simulée avec des taux de fécondité entre 1946 et 1970 égaux à ceux de 1970
- un scénario composite, avec les actifs du scénario central Insee et les seniors du scénario sans baby boom : c'est une approximation du monde sans l'effet du surplus de personnes âgées causé par le babyboom, mais sans l'effet sur le nombre d'actifs de supprimer ce surplus
- un scénario sans allongement d'espérance de vie après 2000, c'est à dire où l'évolution de la population est simulée avec des taux de mortalité après 2000 égaux à ceux de 2000.


## 1. Mise en forme des données

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import plotnine as pn
from plotnine.animation import PlotnineAnimation

from matplotlib import rc
rc('animation', html='html5')

import os

In [ ]:
!pip install xlrd

In [ ]:
def update_population(population, fertility_rates, mortality_rates, migration, male_ratio_at_birth):
    
    new_kids = sum(fertility_rates[age] * population['F'][age] for age in fertility_rates)
    updated_population = {'H': dict(), 'F': dict()}
    
    updated_population['H'][0] = male_ratio_at_birth * new_kids
    updated_population['F'][0] = (1 - male_ratio_at_birth) * new_kids
    
    for sex in ['H', 'F']:
        for age in range(1, 121):
            updated_population[sex][age] = population[sex][age -1] * (1 - mortality_rates[sex][age - 1]) + migration[sex][age]
    
    return updated_population


def simulation(initial_population, fecondite, mortalite, migration, male_ratio_at_birth, start_year, stop_year):
    
    simulated_population = {start_year: initial_population}
    
    for year in range(start_year + 1, stop_year + 1):
        simulated_population[year] = update_population(
            simulated_population[year - 1],
            fecondite[year],
            mortalite[year],
            migration[year],
            male_ratio_at_birth[year]
            )
    
    return simulated_population


def freeze(rates, when, freeze_year):
    if when == 'before':
        frozen_rates = {
            year: (rates[year] if year > freeze_year else rates[freeze_year])
            for year in rates
            }
    if when == 'after':
        frozen_rates = {
            year: (rates[year] if year < freeze_year else rates[freeze_year])
            for year in rates
            }
        
    return frozen_rates

In [ ]:
data_dir = '/kaggle/input/french-demography/'
path = lambda x: os.path.join(data_dir, x)


def parse_projections_deces(filepath, sheet_name):
    projections_deces = pd.read_excel(
        path(filepath),
        sheet_name = sheet_name,
        usecols = "A:FD",
        skiprows = 1,
        nrows = 121
        ) \
        .rename(columns = {str(x) + ' (p)': x for x in [2018, 2019, 2020]}) \
        .set_index("Âge atteint dans l'année") \
        .fillna(method = "ffill") \
        .divide(100000) \
        .to_dict()
    
    return projections_deces
        
        
def parse_projections_fecondite(filepath, sheet_name):
    projections_fecondite = pd.read_excel(
        path(filepath),
        sheet_name = sheet_name,
        usecols = "A:FD",
        skiprows = 1,
        nrows = 36
        ) \
        .set_index("Âge atteint dans l'année ") \
        .divide(10000) \
        .to_dict()
    
    return projections_fecondite


def parse_projections_migration(filepath, sheet_name):
    projections_migration = pd.read_excel(
        path(filepath),
        sheet_name = sheet_name,
        usecols = "A:FD",
        skiprows = 1,
        nrows = 106
        ) \
        .replace(to_replace = '105+', value = 105) \
        .rename(columns = {str(x) + ' (p)': x for x in [2018, 2019, 2020]}) \
        .set_index("Âge atteint dans l'année") \
        .to_dict()

    return projections_migration


def parse_projections_population(filepath, sheet_name):
    projections_population = pd.read_excel(
        path(filepath),
        sheet_name = sheet_name,
        usecols = "A:FD",
        skiprows = 1,
        nrows = 106
        ) \
        .replace(to_replace = '105+', value = 105) \
        .set_index("Âge au 1er janvier") \
        .to_dict()

    return projections_population
    

def parse_observe_fecondite(filepath, sheet_name):
    observe_fecondite = pd.read_excel(
        path(filepath),
        sheet_name = sheet_name,
        usecols = "A:AL",
        skiprows = 4,
        nrows = 77
        ) 
    
    observe_fecondite = \
        observe_fecondite \
        .drop(observe_fecondite.columns[1], axis = 1) \
        .set_index(observe_fecondite.columns[0]) \
        .T \
        .reset_index()

    observe_fecondite['index'] = range(15, 51)

    observe_fecondite = \
        observe_fecondite \
        .set_index('index') \
        .divide(10000) \
        .to_dict()

    return observe_fecondite


def parse_observe_mortalite(filepath, sheet_name):
    observe_mortalite = pd.read_excel(
        path(filepath),
        sheet_name = sheet_name,
        usecols = "A:CX",
        skiprows = 4,
        nrows = 77
        ) 
    
    observe_mortalite = \
        observe_mortalite \
        .set_index(observe_mortalite.columns[0]) \
        .T \
        .reset_index()

    observe_mortalite['index'] = range(0, 101)

    observe_mortalite = \
        observe_mortalite \
        .set_index('index') \
        .divide(100000) \
        .to_dict()

    return observe_mortalite


def parse_observe_population(filepath):
    
    fm_t6 = pd.ExcelFile(path(filepath))

    def parse_sheet(filepath, year):
        if year < 2012:
            rows_to_skip = 8
        elif year == 2012:
            rows_to_skip = 9
        else:
            rows_to_skip = 4

        df = pd.read_excel(
            fm_t6,
            sheet_name = str(year),
            usecols = "B:J" if year < 2020 else 'B:E',
            skiprows = rows_to_skip,
            nrows = 101 if year < 2000 else 106
            )

        h_loc = 2 
        f_loc = 7 if year < 2020 else 3
        return {'H': df.iloc[:, h_loc].to_dict(), 'F': df.iloc[:, f_loc].to_dict()}

    observe_population = {year: parse_sheet(fm_t6, year) for year in range(1945, 2022)}
    
    return observe_population

In [ ]:
def merge_dictionnary(left, split, right):
    return {**{key: value for key, value in left.items() if key < split}, **{key: value for key, value in right.items() if key >= split}}


def series_fill_to_120(age_dict, method):
    age_max = max(age_dict.keys())
    if age_max == 120: 
        return age_dict
    
    fill_value = age_dict[age_max] if method == 'ffill' else age_dict[age_max] / (120 - age_max)
    filled_age_dict = {age : age_dict[age] if age < age_max else fill_value for age in range(0, 121)}
    
    return filled_age_dict


def dict_fill_to_120(year_sex_age_dict, method):
    updated_year_sex_age_dict = {
        year: {sex: series_fill_to_120(year_sex_age_dict[year][sex], method) for sex in ['H', 'F']}
        for year in year_sex_age_dict
        }
    
    return updated_year_sex_age_dict


def reconstruct_migration_year(population_t, population_t_1, fertility_rate_t_1, mortality_rate_t_1, male_ratio_at_birth_t_1):
    zero_migration = {sex: {age: 0 for age in range(0, 121)} for sex in ['H', 'F']}
    population_without_migration = update_population(population_t_1, fertility_rate_t_1, mortality_rate_t_1, zero_migration, male_ratio_at_birth_t_1)
    migration = {sex: {age: population_t[sex][age] - population_without_migration[sex][age] for age in range(1, 121)} for sex in ['H', 'F']}
    return migration

In [ ]:
projections_deces_femmes = parse_projections_deces("00_central.xlsx", 'hyp_mortaliteF')
projections_deces_hommes = parse_projections_deces("00_central.xlsx", 'hyp_mortaliteH')
projections_fecondite = parse_projections_fecondite("00_central.xlsx", 'hyp_fecondite')
projections_migration_hommes = parse_projections_migration("00_central.xlsx", "hyp_soldemigH")
projections_migration_femmes = parse_projections_migration("00_central.xlsx", "hyp_soldemigF")
projections_population_hommes = parse_projections_population("00_central.xlsx", "populationH")
projections_population_femmes = parse_projections_population("00_central.xlsx", "populationF")
observe_fecondite = parse_observe_fecondite("fm_dod_taux_fecondite.xlsx", "Fécondité")
observe_mortalite_hommes = parse_observe_mortalite("fm_dod_quotients_mortalite.xls", "Qmort-H")
observe_mortalite_femmes = parse_observe_mortalite("fm_dod_quotients_mortalite.xls", "Qmort-F")
observe_population = parse_observe_population("fm_t6.fr.xls")

In [ ]:
# Fusion de l'observé et des projections, et compléition jusqu'à 120 ans

fin_observe = 2021

fecondite = merge_dictionnary(observe_fecondite, fin_observe, projections_fecondite)

mortalite_hommes = merge_dictionnary(observe_mortalite_hommes, fin_observe, projections_deces_hommes)
mortalite_femmes = merge_dictionnary(observe_mortalite_femmes, fin_observe, projections_deces_femmes)
mortalite = {annee: {sex: mortalite_hommes[annee] if sex == 'H' else mortalite_femmes[annee] for sex in ['H', 'F']} for annee in mortalite_hommes.keys()}
mortalite = dict_fill_to_120(mortalite, method = 'ffill')

projections_population = {annee: {sex: projections_population_hommes[annee] if sex == 'H' else projections_population_femmes[annee] for sex in ['H', 'F']} for annee in projections_population_femmes.keys()}
population = merge_dictionnary(observe_population, fin_observe, projections_population)
population = dict_fill_to_120(population, method = "split")

male_ratio_at_birth = {year: population[year]['H'][0] / (population[year]['H'][0] + population[year]['F'][0]) for year in population}   

# La migration dans le passée est reconstruite comme la différence entre la population observée et la population observée un an plus tôt, et vieillie d'un an sans migration
reconstructed_migration = {
    year: reconstruct_migration_year(population[year], population[year - 1], fecondite[year - 1], mortalite[year - 1], male_ratio_at_birth[year - 1])
    for year in range(1947, 2101)
    }

projections_migration = {annee: {sex: projections_migration_hommes[annee] if sex == 'H' else projections_migration_femmes[annee] for sex in ['H', 'F']} for annee in projections_migration_hommes.keys()}

migration = merge_dictionnary(reconstructed_migration, fin_observe, projections_migration)
migration = dict_fill_to_120(migration, method = "split")

## 2. Simulations

In [ ]:
start_year = 1946
stop_year = 2100
population_1946 = population[1946]

# scénario insee : scénario central des projections de population insee
scenario_insee = simulation(
    population_1946,
    fecondite,
    mortalite,
    migration,
    male_ratio_at_birth,
    start_year, stop_year
    )

# scénario sans babyboom :
# - on simule l'évolution de la population avec une fécondité entre 1946 et 1970 égale à celle de 1970
# - on simule le scénario central des projections de population insee
# - pour chaque année, on prend les jeunes (< 65 ans) du scénario insee et les vieux (>= 65 ans) du scénario avec une moindre fertilité
# ainsi on a l'effet de l'absence de babyboom sur les âges élevés, sans la réduction du nombre de jeunes que ça provoque à long terme

scenario_no_babyboom = simulation(
    population_1946,
    freeze(fecondite, 'before', 1970),
    mortalite,
    migration,
    male_ratio_at_birth,
    start_year, stop_year
    )

def replace_values_at_age(scenario_1, limit_age, scenario_2):
    updated_scenario_2 = {
        year: {sex: {age: scenario_1[year][sex][age] if age <= limit_age else scenario_2[year][sex][age] for age in range(0, 121)} for sex in ['H', 'F']}
        for year in scenario_2
    }
    
    return updated_scenario_2

scenario_composite = replace_values_at_age(scenario_insee, 65, scenario_no_babyboom)

# scénario frozen_mortality : scénario central des projections de population insee mais la mortalité après 2000 est égale à celle de 2000
scenario_frozen_mortality = simulation(
    population_1946,
    fecondite,
    freeze(mortalite, 'after', 2000),
    migration,
    male_ratio_at_birth,
    start_year, stop_year
    )

scenarios = {
    'scenario_composite': scenario_composite,
    'no_babyboom': scenario_no_babyboom,
    'frozen_mortality': scenario_frozen_mortality,
    'insee': scenario_insee
    }

In [ ]:
# code from https://stackoverflow.com/a/66789625
def flatten_dict(nested_dict):
    res = {}
    if isinstance(nested_dict, dict):
        for k in nested_dict:
            flattened_dict = flatten_dict(nested_dict[k])
            for key, val in flattened_dict.items():
                key = list(key)
                key.insert(0, k)
                res[tuple(key)] = val
    else:
        res[()] = nested_dict
    return res


def nested_dict_to_df(values_dict):
    flat_dict = flatten_dict(values_dict)
    df = pd.DataFrame.from_dict(flat_dict, orient="index")
    df.index = pd.MultiIndex.from_tuples(df.index)
    df = df.unstack(level=-1)
    df.columns = df.columns.map("{0[1]}".format)
    return df


df = nested_dict_to_df(scenarios).reset_index()
df = df.rename(columns = {'level_0': 'scenario', 'level_1': 'annee', 'level_2': 'sexe'})

df = pd.wide_to_long(
    df,
    stubnames = "",
    i = ["scenario", "annee", "sexe"],
    j = "age"
    )

df.columns = ['population']
df = df.reset_index()
df["sexe_scenario"] = df["sexe"] + "_" + df["scenario"]

df = df.fillna(0) # ?

## 3. Comparaison des scénarios

In [ ]:
def plot_pyramid(df, annee):
    temp_df = df.copy()
    temp_df["population"] = (1 - 2 * (temp_df["sexe"] == "F")) * temp_df["population"] 

    breaks = [-50*1e4, -25*1e4, 0, 25*1e4, 50*1e4]
    labels = [str(int(abs(x)/1e3)) + "k" * (x != 0) for x in breaks]
    
    graphe = (
        pn.ggplot(temp_df.query("annee == " + str(annee)))
        + pn.aes(x = "age", y = "population", color = "scenario", group = "sexe_scenario")
        + pn.geom_line()
        + pn.theme_minimal()
        + pn.labs(
            x = "Âge",
            y = "Population",
            color = "Scénario",
            )
        + pn.coord_flip()
        + pn.scale_y_continuous(
            breaks = breaks,
            labels = labels,
            limits = [-51*1e4, 51*1e4]
            )
        + pn.theme(
            aspect_ratio = 1.5,
            subplots_adjust = {'right': 0.7}
            )
        + pn.annotate('text', x = 0, y = 10, label = str(annee))
        )
    
    return graphe
        
pyramids = [plot_pyramid(df, annee) for annee in range(1946, 2101)]
animation = PlotnineAnimation(pyramids, interval = 100)

animation

### Ratio de dépendance des personnes âgées

In [ ]:
def compute_old_age_dependancy_ratio(group):
    dependants = group.query("(age >= 65)")["population"].sum()
    
    actifs = group.query("(age < 65) & (age >= 15)")["population"].sum()
    return dependants / actifs

ratio = pd.DataFrame(
    df \
    .groupby(["scenario", "annee"]) \
    .apply(compute_old_age_dependancy_ratio)
    )

ratio.columns = ["ratio_personnes_agees"]
ratio = ratio.reset_index()

graphe = (
    pn.ggplot(ratio)
    + pn.aes(x = "annee", y = "ratio_personnes_agees", color = "scenario", group = "scenario")
    + pn.geom_line()
    + pn.labs(x = "Année", y = "Ratio de dépendance des personnes âgées", color = "Scénario")
    + pn.theme_minimal()
    )

graphe

In [ ]:
ratio \
    .query("annee in [2020, 2040, 2060]") \
    .pivot(index='scenario', columns='annee', values ='ratio_personnes_agees') \
    .multiply(100) \
    .round()

### Ratio de dépendance

In [ ]:
def compute_dependancy_ratio(group):
    dependants = group.query("(age >= 65) | (age < 15)")["population"].sum()
    
    actifs = group.query("(age < 65) & (age >= 15)")["population"].sum()
    return dependants / actifs

ratio = pd.DataFrame(
    df \
    .groupby(["scenario", "annee"]) \
    .apply(compute_dependancy_ratio)
    )

ratio.columns = ["ratio"]
ratio = ratio.reset_index()

graphe = (
    pn.ggplot(ratio)
    + pn.aes(x = "annee", y = "ratio", color = "scenario", group = "scenario")
    + pn.geom_line()
    + pn.labs(x = "Année", y = "Ratio de dépendance", color = "Scénario")
    + pn.theme_minimal()
    )

graphe

In [ ]:
ratio \
    .query("annee in [2020, 2040, 2060]") \
    .pivot(index='scenario', columns='annee', values ='ratio') \
    .multiply(100) \
    .round()